In [ ]:
import pickle
import os
import json
import re
from glob import glob
json_data = []
paths = glob(r"resp_f1_p1_q1/*.pkl")

In [2]:
for path in paths:
    year, session = re.match(r'(\d{4})_.*_(\d)\.pkl', os.path.basename(path)).groups()
    with open(path, 'rb') as f:
        data = pickle.load(f)
    for index, question in enumerate(data):
        common_question_text_temp =question.split(f'{index+1}. ')[0].strip()
        if common_question_text_temp == '':
            common_question_text = None
            common_materials = None
        else:
            common_question_text = common_question_text_temp.split('\n')[0].strip().split('] ',1)[-1]
            common_materials = common_question_text_temp.split('\n',1)[1].strip() if len(common_question_text_temp.split('\n'))>1 else None

        question_choices = question.split(f'{index+1}. ')[-1].strip()
        question_text=question_choices.split('①')[0].strip()
        choices_merged='① '+question_choices.split('①')[-1].strip()
        choices=choices_merged.split('\n')

        materials = question_text.split('?')[-1].strip()
        if materials == '':
            materials = None
        question_text = question_text.split('?')[0].strip()+'?'

        question_entry = {
            "year": int(year),
            "session": int(session),
            "common_question_text": common_question_text,
            "common_data": common_materials,
            "question_number": index+1,
            "question_text": question_text,
            "data": materials,
            "choices": choices,
        }
        json_data.append(question_entry)

json_output = json.dumps(json_data, ensure_ascii=False, indent=4)

output_file_path = f'opus.json'
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(json_output)